In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from scipy.signal import savgol_filter
import json
from datetime import datetime
import statsmodels.api as sm
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler


interventions = pd.read_csv('interventions.csv')
applemob = pd.read_csv('applemobilitytrends-2020-04-20.csv')
globalmob = pd.read_csv('Global_Mobility_Report.csv')

interventions.set_index('lang', inplace=True)

with open('aggregated_timeseries.json', 'r') as f:
    # Load the JSON data
    d = json.load(f)